<a href="https://colab.research.google.com/github/jasondupree/jasondupree.github.io/blob/main/numerical_mle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EX2



All of this work should be done within a Jupyter notebook. Please format it consistently with the notebooks that we used during the course. In particular, place emphasis on making your comments, code, and plots readable.
Turn in both the .ipynb file and a .pdf version.

## Question One
Let X1, X2, ... Xn be i.i.d. from the Beta(a; b) distribution. Find numerically the maximum likelihood estimators for a and b. Mimic the function gammamle() that we created in
lecture.
The function should also return the covariance matrix for the MLE.
Print the function that you wrote and attach it to your homework, and also demonstrate that
it works by using a few simulations.

Maximum Likelihood Estimation (MLE) is a method for estimating the parameters of a statistical model. Given a set of data and a statistical model with parameters, MLE finds the parameter values that maximize the likelihood function, which measures how likely it is to observe the given data under different parameter values.

Step-by-Step Process

    Define the likelihood function:
    The likelihood function for a normal distribution is based on its probability density function (PDF). For a set of observations, the log-likelihood is often used for numerical stability.

    Use an optimization algorithm:
    To maximize the log-likelihood function, we use an optimization algorithm like scipy.optimize.minimize.

    Estimate the parameters:
    Provide an initial guess for the parameters and use the optimization algorithm to find the parameter values that maximize the log-likelihood.

In [1]:
import numpy as np
from scipy.optimize import minimize
from scipy.stats import beta
from scipy.special import gammaln, psi
from numpy.linalg import inv

def beta_log_likelihood(params, data):
    a, b = params
    if a <= 0 or b <= 0:
        return np.inf  # Return infinity if parameters are non-positive
    n = len(data)
    log_likelihood = (
        n * (gammaln(a + b) - gammaln(a) - gammaln(b))
        + (a - 1) * np.sum(np.log(data))
        + (b - 1) * np.sum(np.log(1 - data))
    )
    return -log_likelihood  # Negative because we will minimize

def beta_mle(data):
    initial_guess = [1, 1]  # Initial guess for a and b
    result = minimize(beta_log_likelihood, initial_guess, args=(data,), method='L-BFGS-B', bounds=[(1e-6, None), (1e-6, None)])
    a_mle, b_mle = result.x

    # Compute the observed Fisher Information Matrix (Hessian)
    hessian_inv = result.hess_inv.todense()

    return a_mle, b_mle, hessian_inv

# Test the function with a few simulations
np.random.seed(0)
data = np.random.beta(2, 5, 1000)
a_mle, b_mle, cov_matrix = beta_mle(data)

print(f"Estimated a: {a_mle}")
print(f"Estimated b: {b_mle}")
print("Covariance matrix:")
print(cov_matrix)

Estimated a: 2.05885302056502
Estimated b: 4.97025499625221
Covariance matrix:
[[0.01275145 0.03818154]
 [0.03818154 0.1410208 ]]


Explanation:

    Data Generation:
        We use np.random.beta to generate sample data from a Beta distribution with known parameters for testing purposes.

    Log-Likelihood Function:
        The function beta_log_likelihood calculates the negative log-likelihood for the Beta distribution. It takes the parameters and data as input and returns the negative log-likelihood value.

    Optimization:
        We use scipy.optimize.minimize with the L-BFGS-B method to find the parameter values that minimize the negative log-likelihood function. The bounds ensure that the parameters remain positive.

    Fisher Information Matrix:
        The covariance matrix of the MLEs is the inverse of the observed Fisher Information Matrix (Hessian). In the case of the L-BFGS-B method, the Hessian inverse is available directly.

    Testing:
        We test the function with simulated data to ensure it works correctly.

## Question Two

Create a new treg() function like the one demonstrated in lecture, but now include the
number of degrees of freedom for the error distribution as an additional parameter to be
estimated via maximum likeihood.
Print the function that you wrote and attach it to your homework, and also demonstrate that
it works by using a few simulations.

The function should also return the covariance matrix for the MLE.

Comment: There is nothing wrong with having a non-integer valued number of degrees of
freedom with the t-distribution.

A treg() function estimates the parameters of a regression model with t-distributed errors, including the number of degrees of freedom for the error distribution. The function will also return the covariance matrix for the MLE.

In [3]:
import numpy as np
from scipy.optimize import minimize
from scipy.stats import t
from numpy.linalg import inv

def neglogliketreg(pars, x, y):
    beta0, beta1, sigma, nu = pars
    if sigma <= 0 or nu <= 0:
        return np.inf
    resid = y - (beta0 + beta1 * x)
    log_likelihood = np.sum(t.logpdf(resid / sigma, df=nu) - np.log(sigma))
    return -log_likelihood  # Negative because we will minimize

def treg(x, y):
    n = len(x)

    # Initial guesses for the parameters
    beta1_init = np.cov(x, y)[0, 1] / np.var(x)
    beta0_init = np.mean(y) - beta1_init * np.mean(x)
    resid = y - (beta0_init + beta1_init * x)
    sigma_init = np.sqrt(np.mean(resid ** 2))
    nu_init = 5  # Initial guess for degrees of freedom

    initial_guess = [beta0_init, beta1_init, sigma_init, nu_init]

    # Perform the optimization
    result = minimize(neglogliketreg, initial_guess, args=(x, y), method='L-BFGS-B', bounds=[(None, None), (None, None), (1e-6, None), (1e-6, 100)])
    beta0_mle, beta1_mle, sigma_mle, nu_mle = result.x

    # Compute the observed Fisher Information Matrix (Hessian)
    hessian_inv = result.hess_inv.todense()

    return (beta0_mle, beta1_mle, sigma_mle, nu_mle), hessian_inv

# Test the function with a few simulations
np.random.seed(0)
x = np.random.uniform(0, 10, 100)
y = 2.5 + 1.3 * x + 5 * t.rvs(df=10, size=len(x))

params, cov_matrix = treg(x, y)
beta0_mle, beta1_mle, sigma_mle, nu_mle = params

print(f"Estimated beta0: {beta0_mle}")
print(f"Estimated beta1: {beta1_mle}")
print(f"Estimated sigma: {sigma_mle}")
print(f"Estimated nu: {nu_mle}")
print("Covariance matrix:")
print(cov_matrix)

Estimated beta0: 0.6166797416082761
Estimated beta1: 1.598952169648803
Estimated sigma: 4.596004313853946
Estimated nu: 79.78849667802736
Covariance matrix:
[[ 9.78279058e-01 -1.51233122e-01  9.66007726e-04  7.78666948e-03]
 [-1.51233122e-01  3.05622361e-02 -2.95575399e-04 -1.00101082e-02]
 [ 9.66007726e-04 -2.95575399e-04  1.09356395e-01  1.53412437e-02]
 [ 7.78666948e-03 -1.00101082e-02  1.53412437e-02  8.90588800e-01]]
